In [1]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np

import onnx
from onnxsim import simplify
import onnxruntime

sys.path.append('../')

from siamfcpp.config.config import cfg as root_cfg
from siamfcpp.config.config import specify_task        
from siamfcpp.engine.builder import build as tester_builder
from siamfcpp.model import builder as model_builder
from siamfcpp.pipeline import builder as pipeline_builder
from siamfcpp.utils import complete_path_wt_root_in_cfg

In [2]:
config = '../models/siamfcpp/test/vot/siamfcpp_alexnet.yaml'
model_path = '../models/snapshots/siamfcpp_alexnet-got/epoch-17.pkl'

In [3]:
exp_cfg_path = os.path.realpath(config)
root_cfg.merge_from_file(exp_cfg_path)
root_cfg.test.track.model.task_model.SiamTrack.pretrain_model_path = model_path

# ROOT_PATH = os.getcwd() 
# root_cfg = complete_path_wt_root_in_cfg(root_cfg, ROOT_PATH)
root_cfg = root_cfg.test
task, task_cfg = specify_task(root_cfg)
task_cfg.freeze()

torch.multiprocessing.set_start_method('spawn', force=True)

In [4]:
model = model_builder.build("track", task_cfg.model)

2023-03-08 16:32:24.771 | INFO     | siamfcpp.model.module_base:update_params:60 - Load pretrained SiamTrack parameters from: ../models/snapshots/siamfcpp_alexnet-got/epoch-17.pkl whose md5sum is 2a050730626f1b083baed91f9a5c4a52


#### backbone_init

In [7]:
class MyModel(nn.Module):
    def __init__(self, main_model):
        super(MyModel, self).__init__()
        self.main_model = main_model
        
    def forward(self, x):
        x = self.main_model.basemodel(x)
        c_z_k = self.main_model.c_z_k(x)
        r_z_k = self.main_model.r_z_k(x)
        
        return c_z_k, r_z_k

In [10]:
small_model = MyModel(model)
inp = torch.Tensor(np.random.uniform(size=(1,3,127,127)).astype(np.float32))

out = small_model(inp)

small_model_path = '../models/onnx/backbone_init.onnx'

torch.onnx.export(small_model, 
                  inp, 
                  small_model_path, 
                  input_names=['input'], output_names=['c_z_k', 'r_z_k'],
                  verbose=True, opset_version=11) 

simplified_model, check = simplify(small_model_path, skip_fuse_bn=False)
onnx.save_model(simplified_model, small_model_path)

graph(%input : Float(1, 3, 127, 127, strides=[48387, 16129, 127, 1], requires_grad=0, device=cpu),
      %130 : Float(96, 3, 11, 11, strides=[363, 121, 11, 1], requires_grad=0, device=cpu),
      %131 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %133 : Float(256, 96, 5, 5, strides=[2400, 25, 5, 1], requires_grad=0, device=cpu),
      %134 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %136 : Float(384, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=0, device=cpu),
      %137 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %139 : Float(384, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=0, device=cpu),
      %140 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %142 : Float(256, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=0, device=cpu),
      %143 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %145 : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=0, device=cpu),
      %146 : Float(256, st

#### backbone

In [12]:
class MyModel(nn.Module):
    def __init__(self, main_model):
        super(MyModel, self).__init__()
        self.main_model = main_model
        
    def forward(self, x):
        x = self.main_model.basemodel(x)
        c_x = self.main_model.c_x(x)
        r_x = self.main_model.r_x(x)
        
        return c_x, r_x

In [13]:
backbone = MyModel(model)
inp = torch.Tensor(np.random.uniform(size=(1,3,303,303)).astype(np.float32))

out = backbone(inp)

small_model_path = '../models/onnx/backbone.onnx'

torch.onnx.export(small_model, 
                  inp, 
                  small_model_path, 
                  input_names=['input'], output_names=['c_x', 'r_x'],
                  verbose=True, opset_version=11) 

simplified_model, check = simplify(small_model_path, skip_fuse_bn=False)
onnx.save_model(simplified_model, small_model_path)

graph(%input : Float(1, 3, 303, 303, strides=[275427, 91809, 303, 1], requires_grad=0, device=cpu),
      %130 : Float(96, 3, 11, 11, strides=[363, 121, 11, 1], requires_grad=0, device=cpu),
      %131 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %133 : Float(256, 96, 5, 5, strides=[2400, 25, 5, 1], requires_grad=0, device=cpu),
      %134 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %136 : Float(384, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=0, device=cpu),
      %137 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %139 : Float(384, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=0, device=cpu),
      %140 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %142 : Float(256, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=0, device=cpu),
      %143 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %145 : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=0, device=cpu),
      %146 : Float(256, s

#### head

In [34]:
inp1 = torch.Tensor(np.random.uniform(size=(1,256,23,23)).astype(np.float32))
inp2 = torch.Tensor(np.random.uniform(size=(1,256,23,23)).astype(np.float32))

out = model.head(inp1, inp2)

small_model_path = '../models/onnx/head.onnx'

torch.onnx.export(model.head, 
                  (inp1,inp2), 
                  small_model_path, 
                  input_names=['input1', 'input2'], output_names=['csl_score', 'ctr_score', 'bbox', 'fea'],
                  verbose=True, opset_version=11) 

simplified_model, check = simplify(small_model_path, skip_fuse_bn=False)
onnx.save_model(simplified_model, small_model_path)

graph(%input1 : Float(1, 256, 23, 23, strides=[135424, 529, 23, 1], requires_grad=0, device=cpu),
      %input2 : Float(1, 256, 23, 23, strides=[135424, 529, 23, 1], requires_grad=0, device=cpu),
      %bi : Float(requires_grad=1, device=cpu),
      %si : Float(requires_grad=1, device=cpu),
      %cls_p5_conv1.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %cls_p5_conv1.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %bbox_p5_conv1.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %bbox_p5_conv1.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %cls_p5_conv2.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %cls_p5_conv2.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %bbox_p5_conv2.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
   